In [158]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
df = pd.read_csv('/content/drive/My Drive/Resources/kingcounty/seattle_sales_cleaned_data.csv')
# Review the DataFrame
df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,...,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,0,1/2/2018,665000,47.559358,-122.317370,SEATTLE,SF 5000,41000,122000,1920,...,0,0,0,0,0,0,0,0,0,2018
1,1,1/9/2018,499000,47.524993,-122.316475,SEATTLE,SF 5000,29000,55000,1907,...,0,0,0,0,0,0,0,0,0,2018
2,2,2/14/2018,675000,47.624203,-122.300762,SEATTLE,SF 5000,97000,132000,1908,...,0,0,0,2,0,0,0,0,0,2018
3,3,2/13/2018,1150000,47.721276,-122.335857,SEATTLE,SF 7200,440000,221000,1920,...,2,0,0,0,0,0,0,0,4,2018
4,4,2/14/2018,705000,47.666210,-122.324618,SEATTLE,SF 5000,81000,72000,1918,...,0,0,0,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,24945,12/8/2022,895000,47.689100,-122.332842,SEATTLE,NR3,440000,327000,1926,...,0,0,0,0,0,0,0,0,0,2022
24934,24946,12/5/2022,1900000,47.595587,-122.291790,SEATTLE,NR3,790000,1432000,1924,...,0,0,0,2,2,0,0,1,0,2022
24935,24947,12/17/2022,1750000,47.629275,-122.307670,SEATTLE,NR3,853000,727000,1908,...,1,0,0,0,0,0,0,0,0,2022
24936,24948,12/15/2022,1250000,47.676524,-122.392174,SEATTLE,NR3,497000,532000,1928,...,0,0,0,0,0,0,0,0,0,2022


In [123]:
df['garb_sqft'].value_counts()

0       18360
200      1084
240       607
220       405
180       339
        ...  
1170        1
950         1
115         1
1370        1
255         1
Name: garb_sqft, Length: 119, dtype: int64

In [124]:
df['gara_sqft'].value_counts()

0       20197
240       302
400       246
440       232
220       227
        ...  
1480        1
900         1
1180        1
880         1
1050        1
Name: gara_sqft, Length: 123, dtype: int64

In [125]:
df['wfnt'].value_counts()

0    24775
6       99
3       48
8       11
5        3
4        2
Name: wfnt, dtype: int64

In [126]:
df = df.drop(columns=['wfnt'])
df = df.drop(columns=['garb_sqft'])
df = df.drop(columns=['gara_sqft'])
df = df.drop(columns=['id'])

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24938 entries, 0 to 24937
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sale_date         24938 non-null  object 
 1   sale_price        24938 non-null  int64  
 2   latitude          24938 non-null  float64
 3   longitude         24938 non-null  float64
 4   city              24938 non-null  object 
 5   zoning            24938 non-null  object 
 6   land_val          24938 non-null  int64  
 7   imp_val           24938 non-null  int64  
 8   year_built        24938 non-null  int64  
 9   sqft_lot          24938 non-null  int64  
 10  sqft              24938 non-null  int64  
 11  grade             24938 non-null  int64  
 12  fbsmt_grade       24938 non-null  int64  
 13  home_condition    24938 non-null  int64  
 14  stories           24938 non-null  float64
 15  beds              24938 non-null  int64  
 16  bath_full         24938 non-null  int64 

In [128]:
df['zoning'].value_counts()
category_mapping = {
    'NR3': 1,
    'NR2': 2,
    'SF 5000': 3,
    'RSL (M)': 4,
    'SF 7200': 5,
    'NR1': 6,
    'SF 9600': 7
}

# Replace categorical values with integer values using the mapping
df['zoning'] = df['zoning'].map(category_mapping)
df['zoning'].value_counts()

1    16716
2     4014
3     2191
4     1293
5      360
6      313
7       51
Name: zoning, dtype: int64

In [129]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_date'] = df['sale_date'].astype('int64') // 10**9
df['sale_date']

0        1514851200
1        1515456000
2        1518566400
3        1518480000
4        1518566400
            ...    
24933    1670457600
24934    1670198400
24935    1671235200
24936    1671062400
24937    1671148800
Name: sale_date, Length: 24938, dtype: int64

In [130]:
df = df.drop(columns=['city'])

In [131]:
df = df.drop(columns=['sale_date'])

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24938 entries, 0 to 24937
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sale_price        24938 non-null  int64  
 1   latitude          24938 non-null  float64
 2   longitude         24938 non-null  float64
 3   zoning            24938 non-null  int64  
 4   land_val          24938 non-null  int64  
 5   imp_val           24938 non-null  int64  
 6   year_built        24938 non-null  int64  
 7   sqft_lot          24938 non-null  int64  
 8   sqft              24938 non-null  int64  
 9   grade             24938 non-null  int64  
 10  fbsmt_grade       24938 non-null  int64  
 11  home_condition    24938 non-null  int64  
 12  stories           24938 non-null  float64
 13  beds              24938 non-null  int64  
 14  bath_full         24938 non-null  int64  
 15  bath_3qtr         24938 non-null  int64  
 16  bath_half         24938 non-null  int64 

In [133]:
summary_stats = df.describe()
print(summary_stats)

         sale_price      latitude     longitude        zoning      land_val  \
count  2.493800e+04  24938.000000  24938.000000  24938.000000  2.493800e+04   
mean   1.064285e+06     47.625447   -122.336537      1.624990  4.870126e+05   
std    6.603739e+05      0.068211      0.043736      1.098562  3.441089e+05   
min    6.500000e+04     47.495701   -122.418376      1.000000  0.000000e+00   
25%    6.990000e+05     47.557221   -122.375839      1.000000  2.910000e+05   
50%    8.900000e+05     47.642224   -122.343144      1.000000  4.320000e+05   
75%    1.250000e+06     47.684239   -122.295313      2.000000  6.050000e+05   
max    1.485000e+07     47.733910   -122.238319      7.000000  7.560000e+06   

            imp_val    year_built       sqft_lot         sqft         grade  \
count  2.493800e+04  24938.000000   24938.000000  24938.00000  24938.000000   
mean   4.873107e+05   1942.581883    6001.867391   1902.27412      7.321718   
std    3.472508e+05     27.603422    3656.752246   

In [147]:
columns = ['sale_price', 'sqft_lot', 'sqft', 'land_val', 'imp_val']

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Calculate IQR for the specified columns
Q1 = df[columns].quantile(0.25)
Q3 = df[columns].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for the specified columns
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers for the specified columns
filtered_df = df[~((df[columns] < lower_bound) | (df[columns] > upper_bound)).any(axis=1)]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()

,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,665000,47.559358,-122.317370,3,41000,122000,1920,2525,1570,6,4,4,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,499000,47.524993,-122.316475,3,29000,55000,1907,6000,1350,5,0,3,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,675000,47.624203,-122.300762,3,97000,132000,1908,5286,1800,7,5,4,1.0,4,3,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,705000,47.666210,-122.324618,3,81000,72000,1918,3825,780,6,0,3,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
4,650000,47.672576,-122.284472,3,93000,100000,1948,6750,830,7,0,4,1.0,2,1,0,0,0,0,2,0,0,0,0,0,0,0,0,2018


In [175]:
y = df["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = df.copy()
X.drop("sale_price", axis=1, inplace=True)

In [176]:
y_scaled[:5]

array([-0.60464624, -0.85602403, -0.589503  ,  0.12980091, -0.54407328])

In [177]:
X.head()

,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,47.559358,-122.317370,3,41000,122000,1920,2525,1570,6,4,4,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,47.524993,-122.316475,3,29000,55000,1907,6000,1350,5,0,3,1.5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,47.624203,-122.300762,3,97000,132000,1908,5286,1800,7,5,4,1.0,4,3,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,47.721276,-122.335857,5,440000,221000,1920,47916,1590,7,0,3,1.5,3,1,0,0,0,0,2,0,0,0,0,0,0,0,4,2018
4,47.666210,-122.324618,3,81000,72000,1918,3825,780,6,0,3,1.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2018


In [178]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

In [179]:
regression_model = LinearRegression()

# Fit the model using training data
regression_model.fit(X_train, y_train)

# Predict values using the trained model
y_pred = regression_model.predict(X_test)

In [180]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.46447419898697034
R-squared: 0.544384068858859


In [181]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [182]:
y_pred2 = rf_model.predict(X_test)

In [199]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred2_original)
rmse_rounded = round(rmse) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse_rounded)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        690000.00        763906.76             73906.76
1       1000000.00       1462756.62            462756.62
2        973000.00        930327.86             42672.14
3       1080000.00       1244272.49            164272.49
4       2025000.00       1991369.47             33630.53
5        755000.00        679570.63             75429.37
6       1565000.00       1655501.50             90501.50
7        879000.00       1178231.82            299231.82
8        650000.00        701607.36             51607.36
9       1750000.00       1682654.15             67345.85
10       881000.00        821140.76             59859.24
11       630000.00        581325.00             48675.00
12       725000.00        740905.01             15905.01
13       745000.00        733107.62             11892.38
14       607000.00        503453.50            103546.50
15      1000000.00        939451.87             60548.13
16      1180000.00       110533